# All oscillation analysis

Brief 1-2 sentence description of notebook.

In [ ]:
# Imports of all used packages and libraries
import sys
import os
import git
import glob
from collections import defaultdict

In [ ]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [ ]:
git_root

In [ ]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
import collections
import itertools
from collections import defaultdict
from itertools import combinations

In [ ]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import mannwhitneyu


In [ ]:
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib.cm as cm
# import seaborn as sns
import colorsys

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
from spectral_connectivity import Multitaper, Connectivity
import spectral_connectivity

In [ ]:
import utilities.helper
import trodes.read_exported

In [ ]:
FONTSIZE = 20

In [ ]:
font = {'weight' : 'medium',
        'size'   : 20}

matplotlib.rc('font', **font)

## Inputs & Data

Explanation of each input and where it comes from.

In [ ]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_3_alone_comp"

In [ ]:
LFP_SPECTRAL_DF = pd.read_pickle("./proc/{}_02_full_spectral.pkl".format(OUTPUT_PREFIX))


# LFP_SPECTRAL_DF = pd.read_pickle("./proc/rce_pilot_2_02_full_spectral_copy.pkl")
# LFP_SPECTRAL_DF = pd.read_pickle("./proc/rce_pilot_2_02_full_spectral.pkl")

In [ ]:
# CHANNEL_MAP_AND_ALL_TRIALS_DF = pd.read_pickle("./proc/full_baseline_and_trial_lfp_traces.pkl")
# MERGED_TRIALS_AND_VIDEO = pd.read_pickle("./proc/trial_SLEAP_and_metadata.pkl")

In [ ]:
theta_band = (4, 12)
gamma_band = (30, 50)
filter_band = (0, 56)

In [ ]:
FONTSIZE=20

In [ ]:
FULL_LFP_TRACES_PKL = "{}_03_spectral_bands.pkl".format(OUTPUT_PREFIX)

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Functions 

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

# Dropping unused columns

In [ ]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# Ideally functions are defined here first and then data is processed using the functions

# function names are short and in snake case all lowercase
# a function name should be unique but does not have to describe the function
# doc strings describe functions not function names




In [ ]:
list(LFP_SPECTRAL_DF.columns)

In [ ]:
dropping_columns = [col for col in LFP_SPECTRAL_DF if "MAD" in col or "modified_zscore" in col or "RMS" in col or "mask" in col or "trace_filtered" in col]




In [ ]:
dropping_columns

In [ ]:
LFP_SPECTRAL_DF = LFP_SPECTRAL_DF.drop(columns=dropping_columns, errors="ignore")

## Filtering frequencies

In [ ]:
list(LFP_SPECTRAL_DF.columns)

In [ ]:
all_frequencies_columns = [col for col in LFP_SPECTRAL_DF if "all_frequencies_all_windows" in col] 

In [ ]:
LFP_SPECTRAL_DF["BLA_LH_graLFP_SPECTRAL_DFr_all_frequencies_all_windows"].iloc[0].shape

In [ ]:
for col in all_frequencies_columns:
    if "power" in col:
        LFP_SPECTRAL_DF[col] = LFP_SPECTRAL_DF.apply(lambda x: x[col][:, (x["power_calculation_frequencies"] >= filter_band[0]) & (x["power_calculation_frequencies"] <= filter_band[1])], axis=1)
LFP_SPECTRAL_DF["power_calculation_frequencies"] = LFP_SPECTRAL_DF["power_calculation_frequencies"].apply(lambda x: x[filter_band[0]: filter_band[1]])


In [ ]:
for col in all_frequencies_columns:
    if "coherence" in col:
        LFP_SPECTRAL_DF[col] = LFP_SPECTRAL_DF.apply(lambda x: x[col][:, (x["coherence_calculation_frequencies"] >= filter_band[0]) & (x["coherence_calculation_frequencies"] <= filter_band[1])], axis=1)
LFP_SPECTRAL_DF["coherence_calculation_frequencies"] = LFP_SPECTRAL_DF["coherence_calculation_frequencies"].apply(lambda x: x[filter_band[0]: filter_band[1]])


In [ ]:
for col in all_frequencies_columns:
    if "granger" in col:
        LFP_SPECTRAL_DF[col] = LFP_SPECTRAL_DF.apply(lambda x: x[col][:, (x["granger_calculation_frequencies"] >= filter_band[0]) & (x["granger_calculation_frequencies"] <= filter_band[1])], axis=1)
LFP_SPECTRAL_DF["granger_calculation_frequencies"] = LFP_SPECTRAL_DF["granger_calculation_frequencies"].apply(lambda x: x[filter_band[0]: filter_band[1]])


In [ ]:
all_frequencies_all_windows

In [ ]:
raise ValueError()

## Filtering by theta and gamma band

- Filtering theta/gamma for coherence

In [ ]:
coherence_columns = [col for col in LFP_SPECTRAL_DF.columns if "coherence" in col and "calculation" not in col and "time" not in col]

In [ ]:
coherence_columns

In [ ]:
for col in coherence_columns:
    brain_region_name = "_".join(col.split("_")[:2])
    print(brain_region_name)
    theta_coherence_col = "{}_coherence_theta".format(brain_region_name)
    gamma_coherence_col = "{}_coherence_gamma".format(brain_region_name)
    LFP_SPECTRAL_DF["{}".format(theta_coherence_col)] = LFP_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col][:, (x["coherence_calculation_frequencies"] >= theta_band[0]) & (x["coherence_calculation_frequencies"] <= theta_band[1])], axis=1), axis=1)
    LFP_SPECTRAL_DF["{}".format(gamma_coherence_col)] = LFP_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col][:, (x["coherence_calculation_frequencies"] >= gamma_band[0]) & (x["coherence_calculation_frequencies"] <= gamma_band[1])], axis=1), axis=1)

In [ ]:
LFP_SPECTRAL_DF.head()

In [ ]:
for col in LFP_SPECTRAL_DF.columns:
    print(col)

- Filtering theta/gamma for granger

In [ ]:
granger_columns = [col for col in LFP_SPECTRAL_DF.columns if "granger" in col and "calculation" not in col and "time" not in col]

In [ ]:
granger_columns

In [ ]:
for col in granger_columns:
    brain_region_name = "-to-".join(col.split("_")[:2])
    print(brain_region_name)
    theta_granger_col = "{}_granger_theta".format(brain_region_name)
    gamma_granger_col = "{}_granger_gamma".format(brain_region_name)
    LFP_SPECTRAL_DF["{}".format(theta_granger_col)] = LFP_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col][:, (x["granger_calculation_frequencies"] >= theta_band[0]) & (x["granger_calculation_frequencies"] <= theta_band[1])], axis=1), axis=1)
    LFP_SPECTRAL_DF["{}".format(gamma_granger_col)] = LFP_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col][:, (x["granger_calculation_frequencies"] >= gamma_band[0]) & (x["granger_calculation_frequencies"] <= gamma_band[1])], axis=1), axis=1)

In [ ]:
LFP_SPECTRAL_DF.head()

- Filtering theta/gamma for power

In [ ]:
power_columns = [col for col in LFP_SPECTRAL_DF.columns if "power" in col and "calculation" not in col and "time" not in col]

In [ ]:
power_columns

In [ ]:
LFP_SPECTRAL_DF["power_calculation_frequencies"].apply(lambda x: (x >= theta_band[0]) & (x <= theta_band[1])).iloc[0]

In [ ]:
for col in power_columns:
    # brain_region_name = col.split("_")[0]
    brain_region_name = col.split("power")[0]
    brain_region_name = brain_region_name.strip("_")
    print(brain_region_name)
    theta_power_col = "{}_power_theta".format(brain_region_name)
    gamma_power_col = "{}_power_gamma".format(brain_region_name)
    LFP_SPECTRAL_DF["{}".format(theta_power_col)] = LFP_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col][:, (x["power_calculation_frequencies"] >= theta_band[0]) & (x["power_calculation_frequencies"] <= theta_band[1])], axis=1), axis=1)
    LFP_SPECTRAL_DF["{}".format(gamma_power_col)] = LFP_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col][:, (x["power_calculation_frequencies"] >= gamma_band[0]) & (x["power_calculation_frequencies"] <= gamma_band[1])], axis=1), axis=1)

In [ ]:
LFP_SPECTRAL_DF.head()

In [ ]:
LFP_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [ ]:
for col in LFP_SPECTRAL_DF.columns:
    print(col)

In [ ]:
raise ValueError()